# TensorFlow Graph für OpenVino in IR Format konvertieren

## Einstellungen

In [6]:
import tensorflow as tf
import os

# 
# workspace_dir = '/home/manuel/Bachelor_Arbeit/TensorFlow/workspace'

# pfad zu trainiertem dataset (enthällt trainings output)
# zB:
dataset_dir = os.path.join('/home/manuel/Bachelor_Arbeit/TensorFlow/workspace/OI_Animals_Augmented_9_3000')

# dataset = 'OI_Animals_LessAug_9_3000'

# name des trainings output ordners
model_output = 'faster_rcnn_inception_v2_coco_2018_01_28_outdropout'

# zu exporttierenden checkpoint festlegen:
checkpoint = 479766
is_faster_rcnn = True
is_inception = True

batch_size = 1
channel = 3

if is_faster_rcnn:
    img_size = 600
    json_config_file = '/opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/faster_rcnn_support_api_v1.14.json'
    pipeline = os.path.join(dataset_dir, model_output, 'faster_rcnn_inception_v2_coco.config')
else:
    img_size = 300
    json_config_file = '/opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/ssd_support_api_v1.14.json'
    if is_inception:
        pipeline = os.path.join(dataset_dir, model_output, 'ssd_inception_v2_coco.config')
    else:
        pipeline = os.path.join(dataset_dir, model_output, 'ssd_mobilenet_v2_coco.config')

    
pb_file = os.path.join(dataset_dir, model_output, 'exported/frozen_inference_graph.pb')

plugin_device = 'MYRIAD'

data_type = 'FP16'

input_shape_str = '['+ str(batch_size) +',' + str(img_size) + ',' + str(img_size) + ',' + str(channel) + ']'

output_dir = os.path.join(dataset_dir, model_output, 'open_vino')

mo_tf_path = '/opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py'


## frozen graph exportieren

In [7]:
checkpoint_str = os.path.join(dataset_dir, model_output, 'model.ckpt-' + str(checkpoint))
exported = os.path.join(dataset_dir, model_output, 'exported')

!python3 /home/manuel/Bachelor_Arbeit/TensorFlow/models/object_detection/export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path $pipeline \
    --trained_checkpoint_prefix $checkpoint_str \
    --output_directory $exported

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

W0308 04:00:00.160966 139878579812160 ag_logging.py:145] Entity <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7f37c82eb0f0>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7f37c82eb0f0>>: AssertionError: Bad argument number for Name: 3, expecting 4
W0308 04:00:00.408336 139878579812160 ag_logging.py:145] Entity <bound method Pooling2D.call of <tensorflow.python.layers.pooling.MaxPooling2D object at 0x7f37c839cc50>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause:

W0308 04:00:00.830011 139878579812160 ag_logging.py:145] Entity <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7f37c83b16a0>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7f37c83b16a0>>: AssertionError: Bad argument number for Name: 3, expecting 4
W0308 04:00:00.897611 139878579812160 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f37c81c9cf8>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: conv

W0308 04:00:01.224167 139878579812160 ag_logging.py:145] Entity <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7f37c8194438>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7f37c8194438>>: AssertionError: Bad argument number for Name: 3, expecting 4
W0308 04:00:01.306444 139878579812160 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f37c825ec50>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: conv

W0308 04:00:01.815758 139878579812160 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f37c82ee978>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f37c82ee978>>: AssertionError: Bad argument number for Name: 3, expecting 4
W0308 04:00:01.850199 139878579812160 ag_logging.py:145] Entity <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7f37c3f7fbe0>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Batch

...

W0308 04:00:13.066934 139878579812160 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f37c1949748>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f37c1949748>>: AssertionError: Bad argument number for Name: 3, expecting 4
W0308 04:00:13.092495 139878579812160 ag_logging.py:145] Entity <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7f37c192f198>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Batch

W0308 04:00:13.489845 139878579812160 ag_logging.py:145] Entity <bound method Pooling2D.call of <tensorflow.python.layers.pooling.MaxPooling2D object at 0x7f37c1949748>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Pooling2D.call of <tensorflow.python.layers.pooling.MaxPooling2D object at 0x7f37c1949748>>: AssertionError: Bad argument number for Name: 3, expecting 4
W0308 04:00:13.584320 139878579812160 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f37c192f1d0>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Conv.call of <tensorf

276 ops no flops stats due to incomplete shapes.
Parsing Inputs...
Incomplete shape.

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          .*BatchNorm.*,.*Initializer.*,.*Regularizer.*,.*BiasAdd.*
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================
Incomplete shape.

Doc:
scope: The nodes in the model graph are organiz


W0308 04:00:15.636759 139878579812160 deprecation_wrapper.py:119] From /home/manuel/Bachelor_Arbeit/TensorFlow/models/object_detection/exporter.py:432: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.

2020-03-08 04:00:16.770571: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-03-08 04:00:16.810900: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2699905000 Hz
2020-03-08 04:00:16.811163: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x58eed70 executing computations on platform Host. Devices:
2020-03-08 04:00:16.811187: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
Instructions for updating:
Use standard file APIs to check for files with this prefix.
W0308 04:00:16.812264 139878579812160 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/te

INFO:tensorflow:SavedModel written to: /home/manuel/Bachelor_Arbeit/TensorFlow/workspace/OI_Animals_Augmented_9_3000/faster_rcnn_inception_v2_coco_2018_01_28_outdropout/exported/saved_model/saved_model.pb
I0308 04:00:26.212902 139878579812160 builder_impl.py:421] SavedModel written to: /home/manuel/Bachelor_Arbeit/TensorFlow/workspace/OI_Animals_Augmented_9_3000/faster_rcnn_inception_v2_coco_2018_01_28_outdropout/exported/saved_model/saved_model.pb

W0308 04:00:26.249691 139878579812160 deprecation_wrapper.py:119] From /home/manuel/Bachelor_Arbeit/TensorFlow/models/object_detection/utils/config_util.py:188: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.

INFO:tensorflow:Writing pipeline config file to /home/manuel/Bachelor_Arbeit/TensorFlow/workspace/OI_Animals_Augmented_9_3000/faster_rcnn_inception_v2_coco_2018_01_28_outdropout/exported/pipeline.config
I0308 04:00:26.249951 139878579812160 config_util.py:190] Writing pipeline config file to /home/manuel/Ba

##  in openvino IR Format konvertieren

In [8]:
!python3 {mo_tf_path} \
--input_model {pb_file} \
--tensorflow_use_custom_operations_config {json_config_file} \
--input_shape {input_shape_str} \
--data_type {data_type} \
--input image_tensor \
--tensorflow_object_detection_api_pipeline_config {pipeline} \
--output_dir {output_dir}


[ WARNING ]  Use of deprecated cli option --tensorflow_use_custom_operations_config detected. Option use in the following releases will be fatal. Please use --transformations_config cli option instead
Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/manuel/Bachelor_Arbeit/TensorFlow/workspace/OI_Animals_Augmented_9_3000/faster_rcnn_inception_v2_coco_2018_01_28_outdropout/exported/frozen_inference_graph.pb
	- Path for generated IR: 	/home/manuel/Bachelor_Arbeit/TensorFlow/workspace/OI_Animals_Augmented_9_3000/faster_rcnn_inception_v2_coco_2018_01_28_outdropout/open_vino
	- IR output name: 	frozen_inference_graph
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	image_tensor
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	[1,600,600,3]
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- Enable 